# Projet CentraleSupelec - Dreem 
# Machine Learning course 3A OBT
## Alexis Tuil et Adil Bousfiha
### Approche Machine Learning Classique
### Preprocessing + Features Extraction + Random Forest

# Load useful libraries

In [ ]:
import os
import math
import numpy as np
from scipy.stats import moment, skew, kurtosis
from scipy.signal import butter, lfilter, freqz, freqs
import h5py
from pyeeg import dfa, hurst
from tqdm import tqdm

# Define utils fonctions

In [3]:
CHUNK_SIZE = 1000

# frequence pour la decomposition du signal eeg
"""In order to extract Energy features from our signals, 
    we filter each eeg between the following frequencies 
    to extract the associated wave type : 
    alpha [8–13 Hz], beta [12–24 Hz], theta [4–8 Hz], delta [0.5–2 Hz],
    """
frequencies = [ [8,13],
                [12,24],
                [4,8],
                [0.5,2]]

order = 4 # ordre du filtre
fs = 50.0 # frequence du filtre

In [1]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [4]:
def compute_features_accel(df):
    res = [[l.var() , skew(l), kurtosis(l), min_max_d(l)] for l in df]
    
    return res

def get_stats(l, with_pyeeg=True):
    """Compute statistis for the array argument
       return an array 
       l : numpy 1D array
    """
    if with_pyeeg:
        return [l.var() , skew(l), kurtosis(l), min_max_d(l), energy(l), dfa(l), hurst(l)]
    else:
        return [l.var() , skew(l), kurtosis(l), min_max_d(l), energy(l)]

def min_max_d(l):
    """Compute the min-max distance of the array argument
        return a scalar
        l : numpy 1D array
    """
    a2 = math.pow(max(l) - min(l), 2)
    b2 = math.pow(np.argmax(l) - np.argmin(l), 2) 
    
    return math.sqrt( a2 + b2 )

def energy(l):
    """ Compute the energy of the array argument
        return a scalar
        l : numpy 1D array
    """
    return sum(l*l)

##### On définit des fonctions utiles qui filtrent le signal échantilloné à la fréquence fs au travers d'un filtre passe-bande d'ordre order, et de bande passante [low, high]

In [5]:
def butter_lowpass(low, high, fs=fs, order=order):
    nyq = 0.5 * fs
    normal_low = low / nyq
    normal_high = high / nyq
    #DEBUG #print('butter_lowpass: %s %s' % (normal_low, normal_high))
    b, a = butter(order, (normal_low, normal_high), btype='band', analog=False)
    return b, a

def butter_lowpass_filter(data, low, high, fs=fs, order=order):
   #DEBUG #print('butter_lowpass_filter: %s %s' % (low, high))
    b, a = butter_lowpass(low, high)
    y = lfilter(b, a, data)
    return y

In [ ]:
def wave_extraction(l, freq, order=order, fs=fs):
    """ Extract the wave with frequence range freq from l
        return an array
        l : numpy 1D array
        freq : array of size 2 """
    #DEBUG #print('wave_extraction: %s' % (freq))
    y = butter_lowpass_filter(l, freq[0], freq[1])
    return(y)

# Feature Extraction

In [ ]:
data = h5py.File('all/train.h5', 'r')
names = ['eeg_1', 'eeg_2', 'eeg_3', 'eeg_4', 'eeg_5', 'eeg_6', 'eeg_7']

## Features sur les raw EEG

##### Extraction des features pour les Raw eeg:
- min-max distance
- Moment d'ordre 2
- Energy
- Variance
- Kurtosis
- Skewness

In [ ]:
for eeg_name in names:
    res = []
    for chunk in tqdm(chunks(data[eeg_name], CHUNK_SIZE)):
        for i in tqdm(range(len(chunk))):
            temp = [stats for freq in frequencies for stats in get_stats(chunk[i,:], with_pyeeg=False)]
            res.append(temp)
            np.savez("features/backup_stats_on_raw_" + eeg_name + "_train", res)

    np.savez("features/stats_on_raw_" + eeg_name + "_train", res)

##### On fait la moyenne sur variance, skew et kurtosis

In [ ]:
# load all matrices
mat1 = np.load("features/stats_on_raw_" + eeg_1 + "_train.npz")['arr_0'][,:3]
mat2 = np.load("features/stats_on_raw_" + eeg_2 + "_train.npz")['arr_0'][,:3]
mat3 = np.load("features/stats_on_raw_" + eeg_3 + "_train.npz")['arr_0'][,:3]  
mat4 = np.load("features/stats_on_raw_" + eeg_4 + "_train.npz")['arr_0'][,:3]  
mat5 = np.load("features/stats_on_raw_" + eeg_5 + "_train.npz")['arr_0'][,:3] 
mat6 = np.load("features/stats_on_raw_" + eeg_6 + "_train.npz")['arr_0'][,:3]  
mat7 = np.load("features/stats_on_raw_" + eeg_7 + "_train.npz")['arr_0'][,:3]

X = np.hstack((mat1, mat2, mat3, mat4, mat5, mat6, mat7))

# compute the mean
X_mean = np.vstack((np.mean(X[:,[0+0,0+3,0+6,0+9,0+12,0+15,0+18]], axis=1),
                    np.mean(X[:,[1+0,1+3,1+6,1+9,1+12,1+15,1+18]], axis=1),
                    np.mean(X[:,[2+0,2+3,2+6,2+9,2+12,2+15,2+18]], axis=1))).transpose()

np.savez('features/stats_mean_train', X_mean)

## Features sur les signaux EEG filtrés

##### On décompose chaque 30 sec epoch eeg en 4 sous signaux, puis on calcul les features statistiques pour chacun des sous signaux

In [ ]:
for eeg_name in names:
    res = []
    for chunk in tqdm(chunks(data[eeg_name], CHUNK_SIZE)):
        for i in tqdm(range(len(chunk))):
            temp = [stats for freq in frequencies for stats in get_stats(wave_extraction(chunk[i,:], freq))]
            res.append(temp)
            np.savez("features/backup_stats_on_filter_" + eeg_name + "_train", res)

    np.savez("features/stats_on_filter_" + eeg_name + "_train", res)

## Features sur les signaux de l'accelerometre 3D (x, y z)

##### On extrati 4 statistiques:
- Variance
- Skewness
- Kurtosis
- Min-max distance

In [ ]:
x = [i for chunk in chunks(data['accelerometer_x']) for i in compute_features_accel(chunk)]
y = [i for chunk in chunks(data['accelerometer_y']) for i in compute_features_accel(chunk)]
z = [i for chunk in chunks(data['accelerometer_z']) for i in compute_features_accel(chunk)]

np.savez('features/stats_accel_train', np.hstack((x, y, z)))

## Combine all features

In [ ]:
mat2 = np.load('features/MMD_train.npz')['arr_0'] # (, 7)
mat3 = np.load('features/M2_train.npz')['arr_0'] #(, 7)
mat4 = np.load('features/stats_mean_train.npz')['arr_0'] #(, 3)
mat5 = np.load('features/energy_train.npz')['arr_0'] #(, 42)
mat6 = np.load('features/stats_accel_train.npz')['arr_0'] #(, 12)
mat7 = np.load('features/stats_on_filter_eeg_1_train.npz')['arr_0'] #(, 28)
mat8 = np.load('features/stats_on_filter_eeg_2_train.npz')['arr_0'] #(, 28)
mat9 = np.load('features/stats_on_filter_eeg_3_train.npz')['arr_0'] #(, 28)
mat10 = np.load('features/stats_on_filter_eeg_4_train.npz')['arr_0'] #(, 28)
mat11 = np.load('features/stats_on_filter_eeg_5_train.npz')['arr_0'] #(, 28)
mat12 = np.load('features/stats_on_filter_eeg_6_train.npz')['arr_0'] #(, 28)
mat13 = np.load('features/stats_on_filter_eeg_7_train.npz')['arr_0'] #(, 28)
X = np.hstack((mat2, mat3, mat4, mat5, mat6, mat7, mat8, mat9, mat10, mat11, mat12, mat13))
# X size : (, 267)

In [ ]:
y = pd.read_csv('all/train_y.csv').sleep_stage

# Classification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

## Choice of the classifier
voir en fin de notebook

In [ ]:
p = CompareClassifiers(X,y)
p.run()

## Parameters Tuning

In [ ]:
rf = RandomForestClassifier(random_state = 42)
# Number of trees in random forest
n_estimators = [20]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'class_weight':['balanced', 'balanced_subsample', None]}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, 
                               scoring="f1_macro", verbose=1, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

In [ ]:
print(rf_random.best_params_)
print(rf_random.best_score_)

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [20],
    'max_features': ['auto'],
    'min_samples_leaf': [4, 10, 15],
    'min_samples_split': [10, 20, 30],
    'n_estimators': [20],
    'class_weight':['balanced_subsample']
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 1, scoring='f1_macro')

grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_params_)
print(grid_search.best_score_)

## Test

In [ ]:
clf = RandomForestClassifier(n_estimators = 200, 
                             min_samples_split = 10,
                             min_samples_leaf = 4,
                             max_features = 'auto',
                             max_depth = 20,
                             class_weight  = 'balanced_subsample',
                             bootstrap = False)
clf.fit(X_train, y_train)
print(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="macro"))

## Train on all data

In [ ]:
clf.fit(X, y)

## Predict on the real test set

In [ ]:
"""features has been extracted from the test set the same way than the train set
   we don't show it again here
"""
mat2 = np.load('features/MMD_test.npz')['arr_0']
mat3 = np.load('features/M2_test.npz')['arr_0']
mat4 = np.load('features/stats_test_mean.npz')['arr_0']
mat5 = np.load('features/energy_test.npz')['arr_0']
mat6 = np.load('features/stats_accel_test.npz')['arr_0']
np.nan_to_num(mat6, copy=False)
mat7 = np.load('features/stats_on_filter_eeg_1_test.npz')['arr_0']
mat8 = np.load('features/stats_on_filter_eeg_2_test.npz')['arr_0']
mat9 = np.load('features/stats_on_filter_eeg_3_test.npz')['arr_0']
mat10 = np.load('features/stats_on_filter_eeg_4_test.npz')['arr_0']
mat11 = np.load('features/stats_on_filter_eeg_5_test.npz')['arr_0']
mat12 = np.load('features/stats_on_filter_eeg_6_test.npz')['arr_0']
mat13 = np.load('features/stats_on_filter_eeg_7_test.npz')['arr_0']

X_true_test = np.hstack((mat2, mat3, mat4, mat5, mat6, mat7, mat8, mat9, mat10, mat11, mat12, mat13))

In [ ]:
from utils.submit import Submit
res = Submit(clf.predict(X_true_test))
res.submit('submissions/to_submit.csv')

results on submit : 0.63203

# Annexe

### Class pour comparer les algorithmes de classification

In [ ]:
# Compare Algorithms
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier


class CompareClassifiers:
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def run(self):
        # prepare configuration for cross validation test harness
        seed = 7
        # prepare models
        models = []
        models.append(('LR', LogisticRegression(random_state=0, solver='lbfgs')))
        models.append(('BAG', BaggingClassifier(n_estimators=20, n_jobs=-1)))
        models.append(('GB', GradientBoostingClassifier(n_estimators=20,)))
        models.append(('SVC', SVC(gamma='auto')))
        models.append(('RF', RandomForestClassifier(n_estimators=100, random_state=0, n_jobs = -1, bootstrap=True, max_features='sqrt')))
        
        # evaluate each model in turn
        results = []
        names = []
        scoring = 'f1_macro'
        for name, model in models:
            print("running for %s" % (name))
            kfold = model_selection.KFold(n_splits=10, random_state=seed)
            cv_results = model_selection.cross_val_score(model, self.X, self.y, cv=kfold, scoring=scoring, verbose=2)
            results.append(cv_results)
            names.append(name)
            msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
            print(msg)
        # boxplot algorithm comparison
        fig = plt.figure()
        fig.suptitle('Algorithm Comparison')
        ax = fig.add_subplot(111)
        ax.set(ylabel='F1 score')
        plt.boxplot(results)
        ax.set_xticklabels(names)
        plt.show()
        fig.savefig('compare_clf.png')

### Class pour créer le csv de submit

In [ ]:

class Submit:
    def __init__(self, y_pred, y_true=None):
        self.y_pred = y_pred
        self.y_true = y_true
        self.results = pd.DataFrame()

    def score(self, average='weighted'):
        if self.y_true == None:
            raise ValueError("true labels not found")
        return f1_score(self.y_true, self.y_pred, average=average)

    def submit(self, filename):
        res = pd.DataFrame()
        res['id'] = range(len(self.y_pred))
        res['sleep_stage'] = self.y_pred
        res.set_index('id').to_csv(filename)
        return "csv created at %s" % filename